# 生成数据

In [59]:
import numpy as np
import pickle
import pandas as pd
import warnings
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
import os

In [305]:
submit = pd.read_csv('data/Antai_AE_round1_submit_20190715.csv', header=None)
item =  pickle.load(open('item_attr.pkl', 'rb'), encoding='iso-8859-1')
test =  pickle.load(open('test.pkl', 'rb'), encoding='iso-8859-1')
train =  pickle.load(open('train.pkl', 'rb'), encoding='iso-8859-1')

In [306]:
item[:3]

,item_id,cate_id,store_id,item_price
0,240607,1495,12239,1
1,285232,1498,12239,1
2,113669,1503,12239,1


In [4]:
train[:4]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend
0,xx,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,1,4,0
1,xx,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,1,4,0
2,yy,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,1,0,0
3,xx,8131786,3,2018-08-31 06:00:19,9,6,31,8,2018,8-31,1,4,0


In [92]:
test[:3]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend
0,yy,1061132,189045,2018-07-17 07:43:40,18,7,17,7,2018,7-17,1,1,0
1,yy,1081430,10713670,2018-07-17 09:43:15,8,9,17,7,2018,7-17,1,1,0
2,yy,1918135,10546322,2018-07-17 11:11:34,9,11,17,7,2018,7-17,1,1,0


对于训练集，有多少商品没有出现在商品列表中

In [21]:
train_lose_item = set(train['item_id'].unique()).difference(set(item['item_id'].unique()))
test_lose_item = set(test['item_id'].unique()).difference(set(item['item_id'].unique()))
print('训练集中商品数 %s, 商品表中商品数 %s'%(train['item_id'].unique().shape[0], item['item_id'].unique().shape[0]))
print('训练集中不在商品表中的商品数：%s'%(len(train_lose_item)))
print('测试集中不在商品表中的商品数：%s'%(len(test_lose_item)))

训练集中商品数 2812048, 商品表中商品数 2832669
训练集中不在商品表中的商品数：7733
测试集中不在商品表中的商品数：313


如何处理这样的缺失值？可能直接删除

测试集影响不大，因为本身就是预测测试集中用户下一个可能购买的商品，但是对于训练集，可能意味着就缺少了该条信息，所以最好查看一下这样的商品数量有多少，其所占比重又是多少

In [26]:
list(train_lose_item)[:3]

[5799939, 6914056, 1474573]

In [34]:
# 高频item_id
temp = train.drop_duplicates(subset=['buyer_admin_id','item_id'], keep='first')
item_cnts = temp.groupby(['item_id']).size().reset_index()
item_cnts.columns = ['item_id','cnts']
item_cnts = item_cnts.sort_values('cnts', ascending=False)
items = item_cnts['item_id'].values.tolist()

缺失商品在训练集中的情况

In [60]:
idx = []
for item in tqdm(list(train_lose_item)):
    idx.append(np.where(item_cnts['item_id'].values == item)[0][0])

100%|██████████| 7733/7733 [00:14<00:00, 551.91it/s]


In [71]:
cnts = item_cnts.iloc[idx,:].sort_values('cnts',ascending=False)

In [91]:
print('缺失商品数量在训练集中的均值为:%s'%cnts.iloc[:,1].mean())
print('缺失商品数量在训练集中的中值为:%s'%cnts.iloc[:,1].median())
print('缺失商品数量在训练集中的最大值为:%s'%cnts.iloc[:,1].max())
print('缺失商品数量在训练集中是三位数有:%s'%(sum(cnts.iloc[:,1]>99)) )
print('缺失商品数量在训练集中是二位数有:%s'%(sum(cnts.iloc[:,1]>9)) )
print('缺失商品数量总共有:%s'%(sum(cnts.iloc[:,1])) )

缺失商品数量在训练集中的均值为:2.4245441613862666
缺失商品数量在训练集中的中值为:1.0
缺失商品数量在训练集中的最大值为:642
缺失商品数量在训练集中是三位数有:17
缺失商品数量在训练集中是二位数有:159
缺失商品数量总共有:18749


In [90]:
cnts.iloc[:17]

,item_id,cnts
2318262,10651268,642
2194144,10045561,498
2676935,12227154,353
2728607,12426042,341
2705406,12330663,298
2509168,11533456,285
2712338,12357479,284
1337825,6087123,266
2747008,12497613,228
2641719,12093782,181


+ 小结：

在训练集中，大约有18749条数据中的 item没有出现在商品表中，也就是这在使用这些数据进行训练时，缺少item 的特征

不用看测试集，因为不是使用用户对应item 进行预测的，预测的方式，是使用测试集中item及其之前消费信息，作为用户特征，然后结合候选item 进行预测

测试集中的用户和训练集中的用户有多少是重合的呢?

In [22]:
print('训练集中用户数 %s, 测试集中用户数 %s'%(train['buyer_admin_id'].unique().shape[0], test['buyer_admin_id'].unique().shape[0]))
coincide_buyer = set(train['buyer_admin_id'].unique()).intersection(set(test['buyer_admin_id'].unique()))
print('同时在训练集和测试集中出现的用户数 %s'%(len(coincide_buyer)))

训练集中用户数 809213, 测试集中用户数 11398
同时在训练集和测试集中出现的用户数 6


所以，利用训练集中的用户数据训练得到模型之后，输入测试集用户数据，结合前n个candidate商品，得到最后评分并进行排名，取前30个

# 热销商品

需要定义一下流行和非流行商品，针对训练集和测试集应该各自计算，这个流行与非流行是相对的

In [119]:
popular_item_intrain = train.groupby(['item_id'])['count'].sum().reset_index().sort_values('count',ascending=False)

In [176]:
popular_item_idx_train = popular_item_intrain.iloc[:,0][popular_item_intrain.iloc[:,1].cumsum() <= int(train.shape[0] * 0.15)].values
print('热销商品的数量为：%s'%(popular_item_idx_train.shape[0]))

热销商品的数量为：3220


In [177]:
popular_item_idx_train[:3]

array([12691565, 12824199, 12673275])

# 用户特征提取

首先要根据buyer_id进行聚类，得到其购买的item_id，然后由这些item_id提取得到相关信息，作为该buyer的特征，训练集是如此，测试集也是要先进行相应的操作。现在可以知道的是每个buyer都至少有7条商品信息

三个方面：

    1. 商品
    
    2. 种类类别
    
    3. 商店类别

+ 首先对于训练集

In [250]:
length = len(list(train_lose_item))
all_drop_index= []
for i,item in enumerate(list(train_lose_item)):
    drop_index = train_copy[train_copy['item_id'] == item].index[:]
    all_drop_index.extend(list(drop_index.values))
    print('%.4f'%(i*100/length),end='\r')
#     train_copy.drop(drop_index)

In [481]:
train_filter = train.drop(all_drop_index).reset_index(drop=True)  # 去掉item不在商品表中的数据
train_filter = pd.merge(train_filter,item, how='left',on='item_id') # 将商品表中的数据添加到原始dataframe中

In [482]:
#将下单时间作分类，分成三个时间段：0-8:1, 8-16:2, 16-24；3 
a = train_filter['hour'].to_list()
b = []
for i in range(len(a)):
    if a[i]>=0 and a[i]<8:
        b.append(1)
    elif a[i]>=8 and a[i]<16:
        b.append(2)
    elif a[i]>=16 and a[i]<24:
        b.append(3)
#     print('%.2f'%(100*i/len(a)),end='\r')

In [483]:
train_filter['time_cluster'] = b

In [696]:
time_cluster = pd.get_dummies(train_filter['time_cluster'])
train_filter = pd.concat([train_filter,time_cluster],axis=1)
train_filter = train_filter.rename(columns={1:'shoptime_1',2:'shoptime_2',3:'shoptime_3'})

In [720]:
time_cluster.sum()

1    5926782
2    4806884
3    2109398
dtype: int64

In [697]:
train_filter[:4]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,...,dayofweek,isweekend,cate_id,store_id,item_price,time_cluster,if_popular,shoptime_1,shoptime_2,shoptime_3
0,1,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,...,4,0,2324,10013,4501,3,0.0,0,0,1
1,1,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,...,4,0,3882,4485,2751,3,0.0,0,0,1
2,2,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,...,0,0,155,8341,656,2,0.0,0,1,0
3,1,8131786,3,2018-08-31 06:00:19,9,6,31,8,2018,8-31,...,4,0,155,8341,656,1,0.0,1,0,0


xx：1， yy：2，zz：3

In [487]:
train_filter['buyer_country_id'][train_filter['buyer_country_id'] == 'xx'] =1
train_filter['buyer_country_id'][train_filter['buyer_country_id'] == 'yy'] =2
train_filter['buyer_country_id'][train_filter['buyer_country_id'] == 'zz'] =3

In [492]:
train_filter[:4]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend,cate_id,store_id,item_price,time_cluster
0,1,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,1,4,0,2324,10013,4501,3
1,1,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,1,4,0,3882,4485,2751,3
2,2,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,1,0,0,155,8341,656,2
3,1,8131786,3,2018-08-31 06:00:19,9,6,31,8,2018,8-31,1,4,0,155,8341,656,1


+ 构造用户画像dataframe，方便后续操作

In [626]:
# 合并热销商品id到训练集中
popular_item_df = pd.DataFrame(columns=['item_id','if_popular'])
popular_item_df['item_id'] = popular_item_idx_train
popular_item_df['if_popular'] = 1
train_filter = pd.merge(train_filter,popular_item_df,how='left',on='item_id')
train_filter['if_popular'].fillna(0, inplace=True)

In [628]:
train_filter[:4]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend,cate_id,store_id,item_price,time_cluster,if_popular
0,1,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,1,4,0,2324,10013,4501,3,0.0
1,1,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,1,4,0,3882,4485,2751,3,0.0
2,2,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,1,0,0,155,8341,656,2,0.0
3,1,8131786,3,2018-08-31 06:00:19,9,6,31,8,2018,8-31,1,4,0,155,8341,656,1,0.0


In [580]:
train_admin_df = pd.DataFrame(columns=['buyer_admin_id',  #用户id,
#                                        'buyer_country_id',  # 国家
                                       'num_shop',  #购买次数，
                                       'num_item',  #购买商品件数，
                                       'pop_unpop_rate',  #热销商品与非热销商品件数的比重，
                                       'item_price_max',  #商品最大价格
                                       'item_price_min',  #商品最小价格
                                       'item_price_mean',  #商品价格均值
                                       'item_price_median', #商品价格中值
                                       'num_item_cate',  #商品类别数
                                       'num_store_cate', #商店类别数
                                       'shoptime_1',  #在0-8时间段购买商品数所占比重
                                       'shoptime_2', #在8-16时间段...
                                       'shoptime_3',  #在16-24时间段...
                                      ])

In [629]:
train_buyer_id = np.sort(train_filter['buyer_admin_id'].unique())

In [630]:
train_admin_df['buyer_admin_id'] = train_buyer_id  # id填充

In [631]:
train_admin_df['num_shop'] = train_filter.groupby(['buyer_admin_id'])['count'].sum().to_list()  #，每个admin购买次数

In [565]:
# 因为有些客户可能不知在一个国家购买东西，所以数据长度上暂时还无法对应
# tmp= pd.merge(train_admin_df,train_filter[['buyer_admin_id','buyer_country_id']].drop_duplicates(),how='right',on=['buyer_admin_id','buyer_country_id'])

购买商品件数

In [632]:
num_item = train_filter[['buyer_admin_id','item_id','count']].drop_duplicates().groupby(['buyer_admin_id'])['count'].sum()
train_admin_df['num_item'] =num_item.to_list()

流行商品的占比

In [652]:
tmp = train_filter.groupby(['buyer_admin_id'])['if_popular'].sum()/train_filter.groupby(['buyer_admin_id'])['count'].sum()
train_admin_df['pop_unpop_rate'] = tmp.to_list()

In [654]:
train_admin_df = train_admin_df.drop(['populat_item_rate'], axis=1)

商品价格

In [657]:
# 总价
train_admin_df['item_price_sum'] = train_filter.groupby(['buyer_admin_id'])['item_price'].sum().to_list()

In [659]:
#最大值
train_admin_df['item_price_max'] = train_filter.groupby(['buyer_admin_id'])['item_price'].max().to_list()
#最小值
train_admin_df['item_price_min'] = train_filter.groupby(['buyer_admin_id'])['item_price'].min().to_list()
#均值
train_admin_df['item_price_mean'] = train_filter.groupby(['buyer_admin_id'])['item_price'].mean().to_list()
#中值
train_admin_df['item_price_median'] = train_filter.groupby(['buyer_admin_id'])['item_price'].median().to_list()

In [737]:
#商品类别数
train_admin_df['num_item_cate'] = train_filter[['buyer_admin_id','cate_id','count']].drop_duplicates().groupby(['buyer_admin_id'])['count'].sum().to_list()

In [738]:
# 商店类别数
train_admin_df['num_store_cate'] = train_filter[['buyer_admin_id','store_id','count']].drop_duplicates().groupby(['buyer_admin_id'])['count'].sum().to_list()

In [724]:
# 各时间段购买商品数量
train_admin_df['shoptime_1'] = train_filter.groupby(['buyer_admin_id'])['shoptime_1'].sum().to_list()
train_admin_df['shoptime_2'] = train_filter.groupby(['buyer_admin_id'])['shoptime_2'].sum().to_list()
train_admin_df['shoptime_3'] = train_filter.groupby(['buyer_admin_id'])['shoptime_3'].sum().to_list()

In [743]:
train_admin_df[:4]

,buyer_admin_id,num_shop,num_item,pop_unpop_rate,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,item_price_sum
0,1,28,27,0.000000,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0,13387
1,29,42,34,0.261905,4710,12,331.523810,146.5,22,32,14.0,20.0,8.0,13924
2,71,10,8,0.300000,6306,637,1854.700000,1175.0,7,7,0.0,2.0,8.0,18547
3,81,7,7,0.000000,3599,307,1300.000000,462.0,7,7,1.0,6.0,0.0,9100


商品特征

In [753]:
train_item_df = pd.DataFrame(columns=['item_id','num_sell'])

In [754]:
train_item_df['item_id'] = train_filter['item_id'].unique()
train_item_df['num_sell'] = train_filter.groupby(['item_id'])['count'].sum().to_list()

In [756]:
train_item_df = pd.merge(train_item_df, item, on='item_id',how='left')

In [759]:
train_item_df[:10]

,item_id,num_sell,cate_id,store_id,item_price
0,1,1,2324,10013,4501
1,2,1,3882,4485,2751
2,3,2,155,8341,656
3,5,1,1191,1949,1689
4,6,65,3738,6646,549
5,72,1,537,71,1000
6,73,1,1270,3395,3800
7,74,1,652,18,8685
8,75,6,2319,6677,79
9,81,1,2060,60,208


In [765]:
train_item_df.shape

(2804315, 5)

In [764]:
train_item_df.values[:,0] == [1,2,3,4,5]

False

# Archive

商品，主要看item_id

In [332]:
# 购买次数
onebuyer_features.append(train_onebuyer.shape[0])
print('该用户购买商品的次数为： %s次, 商品件数： %s件'%(train_onebuyer.shape[0],len(train_onebuyer['item_id'].unique()) ))

该用户购买商品的次数为： 17次, 商品件数： 16件


In [319]:
popular_item_idx_train.shape

(3220,)

In [330]:
# 统计热销商品个数与非热销商品个数
popular_item_num = len(set(popular_item_idx_train).intersection(train_onebuyer['item_id'].unique()))  # 没有一件商品是热销商品
print('其中热销商品：%s, 非热销商品 :%s'%(popular_item_num,len(train_onebuyer['item_id'].unique())-popular_item_num))

其中热销商品：0, 非热销商品 :16


In [328]:
#商品价格最大值
item_prices_max = train_onebuyer['item_price'].max()
#商品价格最小值
item_prices_min = train_onebuyer['item_price'].min()
#商品平均值
item_prices_mean = train_onebuyer['item_price'].mean()
#商品中指
item_prices_median = train_onebuyer['item_price'].median()
print('该用户购买过的商品中的价格最大值为：%s，最小值：%s, 平均值: %.4f,中值：%s'%(item_prices_max,item_prices_min,item_prices_mean,item_prices_median))

该用户购买过的商品中的价格最大值为：19657，最小值：101, 平均值: 3362.7647,中值：2200.0


In [335]:
print('商品类别数：%s'%len(set(train_onebuyer['cate_id'].unique())))
print('商店类别数：%s'%len(set(train_onebuyer['store_id'].unique())))

商品类别数：12
商店类别数：14


应该计算购买时间段占所有时间段的比重，这样构建出三个特征

In [364]:
tmp = train_onebuyer['time_cluster'].value_counts()/train_onebuyer['time_cluster'].value_counts().sum()

In [368]:
tmp.keys()[0]

1

In [362]:
train_onebuyer['time_cluster'].value_counts().sum()

17

In [359]:
train_onebuyer

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend,cate_id,store_id,item_price,time_cluster
4,xx,9778613,5,2018-08-21 06:01:56,14,6,21,8,2018,8-21,1,1,0,1191,1949,1689,1
193625,xx,9778613,196096,2018-08-29 01:21:11,7,1,29,8,2018,8-29,1,2,0,2015,2950,3951,1
373555,xx,9778613,378267,2018-08-29 09:55:44,5,9,29,8,2018,8-29,1,2,0,1422,24692,600,2
2414390,xx,9778613,2444194,2018-08-05 01:27:33,15,1,5,8,2018,8-5,1,6,1,443,18771,2200,1
2414391,xx,9778613,2444194,2018-08-05 01:27:33,16,1,5,8,2018,8-5,1,6,1,443,18771,2200,1
2688037,xx,9778613,2721428,2018-08-30 10:17:21,1,10,30,8,2018,8-30,1,3,0,7,3000,1091,2
3325144,xx,9778613,3367023,2018-08-29 01:30:44,6,1,29,8,2018,8-29,1,2,0,554,2950,340,1
3352723,xx,9778613,3394931,2018-08-05 01:08:48,17,1,5,8,2018,8-5,1,6,1,3073,5524,2338,1
4499380,xx,9778613,4556768,2018-08-29 10:17:43,4,10,29,8,2018,8-29,1,2,0,2142,44211,2101,2
6195076,xx,9778613,6277387,2018-08-27 03:09:48,11,3,27,8,2018,8-27,1,0,0,2142,53396,2499,1


In [ ]:
np.array()